Convert TIFFs to Zarr
https://pythonhosted.org/python-bioformats/#reading-images

In [ ]:
import math
import numpy as np
import os
import sys
import xmltodict
from zarr.hierarchy import open_group

import javabridge
import bioformats
from omero_zarr import raw_pixels

In [ ]:
javabridge.start_vm(class_path=bioformats.JARS)
# Reduce logging (default is DEBUG)
# https://forum.image.sc/t/python-bioformats-and-javabridge-debug-messages/12578/12
log_level = 'INFO'
rootLoggerName = javabridge.get_static_field("org/slf4j/Logger", "ROOT_LOGGER_NAME", "Ljava/lang/String;")
rootLogger = javabridge.static_call("org/slf4j/LoggerFactory", "getLogger", "(Ljava/lang/String;)Lorg/slf4j/Logger;", rootLoggerName)
logLevel = javabridge.get_static_field("ch/qos/logback/classic/Level", log_level, "Lch/qos/logback/classic/Level;")
javabridge.call(rootLogger, "setLevel", "(Lch/qos/logback/classic/Level;)V", logLevel)

In [ ]:
dataset = '22-06-16_Tc-Squash-eGFP'
image_template = 'TP{index}_Ch0_Ill0_Ang1,2,3,4,5.tif'
maxindex = 212
idr_fileset_dir = '/uod/idr/filesets'
filepath = f'{idr_fileset_dir}/idr0099-jain-beetlelightsheet/20201001-ftp/Akanksha_Jain_{dataset}/{image_template}'

out_dtype = np.uint16
out_dir = os.path.expandvars('tmp/${USER}/python-bioformats2zarr/' + dataset)

Get dimensions from the first image

In [ ]:
f = filepath.format(index=0)
m = xmltodict.parse(bioformats.get_omexml_metadata(f))
p = bioformats.load_image(f, c=0, z=0, t=0, rescale=False)
[size_x, size_y, size_z, size_c, size_t] = [
    int(m['OME']['Image']['Pixels'][f'@Size{f}']) for f in 'XYZCT']
dtype = m['OME']['Image']['Pixels']['@Type']
print([size_x, size_y, size_z, size_c, size_t], dtype)

In [ ]:
zct_list = []
size_t = maxindex + 1
# Keep going until largest dimension is in [96,192)
# https://github.com/ome/omero-cli-zarr/blob/a4339028a69cabc4f9fd214ad4c868ff72781e8d/src/omero_zarr/raw_pixels.py#L74
level_count = int(math.log2((max(size_x, size_y, 96) - 1) / 96)) + 1

# For testing convert just the first few z/t
# size_t = 20
# size_z = 10
def plane_generator():
    for t in range(size_t):
        f = filepath.format(index=t)
        sys.stdout.write(f'{f}\n')
        for c in range(size_c):
            for z in range(size_z):
                if os.path.exists(f):
                    p = bioformats.load_image(f, c=c, z=z, t=0, rescale=False)
                    yield p.astype(out_dtype)
                else:
                    yield None

In [ ]:
os.makedirs(os.path.dirname(out_dir), mode=511, exist_ok=True)
root = open_group(out_dir, mode='w')
levels = raw_pixels.add_raw_image(
    planes=plane_generator(),
    size_x=size_x,
    size_y=size_y,
    size_z=size_z,
    size_c=size_c,
    size_t=size_t,
    level_count=level_count,
    d_type=out_dtype,
    parent=root)
print(f'Levels: {levels}')
raw_pixels.add_group_metadata(root, None, levels)


In [ ]:
print('Finished!')

In [ ]:
javabridge.kill_vm()